# Description

This notebook exports results into other more accessible data formats.

# Modules

In [1]:
from pathlib import Path
import shutil

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import conf
from utils import remove_all_file_extensions

In [2]:
readRDS = ro.r["readRDS"]

In [3]:
saveRDS = ro.r["saveRDS"]

# Settings

In [4]:
OUTPUT_DIR = Path(conf.__file__).parent.parent / "data" / "gls"
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/opt/code/data/gls')

# Get results files

In [5]:
input_filepath = conf.RESULTS["GLS"] / "gls-summary-phenomexcan.pkl.gz"
display(input_filepath)

PosixPath('/opt/data/results/gls/gls-summary-phenomexcan.pkl.gz')

In [6]:
data = pd.read_pickle(input_filepath)

In [7]:
data.shape

(4037817, 5)

In [8]:
data.head()

,phenotype,phenotype_desc,lv,pvalue,fdr
0,AB1_OTHER_VIRAL,Other viral diseases,LV736,0.004725,0.504339
1,AB1_OTHER_VIRAL,Other viral diseases,LV320,0.004848,0.508291
2,AB1_OTHER_VIRAL,Other viral diseases,LV366,0.005306,0.523691
3,AB1_OTHER_VIRAL,Other viral diseases,LV964,0.006106,0.548143
4,AB1_OTHER_VIRAL,Other viral diseases,LV92,0.006565,0.560048


In [9]:
assert not data.isna().any().any()

# Save

## Pickle format

In [10]:
shutil.copy(input_filepath, OUTPUT_DIR)

'/opt/code/data/gls/gls-summary-phenomexcan.pkl.gz'

## RDS format

In [11]:
output_file = remove_all_file_extensions(input_filepath).with_suffix(".rds")
output_file = OUTPUT_DIR / output_file.name
display(output_file)

PosixPath('/opt/code/data/gls/gls-summary-phenomexcan.rds')

In [12]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_r = ro.conversion.py2rpy(data)

In [13]:
data_r

phenotype,phenotype_desc,lv,pvalue,fdr
...,...,...,...,...


In [14]:
saveRDS(data_r, str(output_file))

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f967bda4880> [RTYPES.NILSXP]

In [15]:
# testing: load the rds file again
data_r = readRDS(str(output_file))

In [16]:
with localconverter(ro.default_converter + pandas2ri.converter):
    data_again = ro.conversion.rpy2py(data_r)
    data_again.index = data_again.index.astype(int)

In [17]:
data_again.shape

(4037817, 5)

In [18]:
data_again.head()

,phenotype,phenotype_desc,lv,pvalue,fdr
0,AB1_OTHER_VIRAL,Other viral diseases,LV736,0.004725,0.504339
1,AB1_OTHER_VIRAL,Other viral diseases,LV320,0.004848,0.508291
2,AB1_OTHER_VIRAL,Other viral diseases,LV366,0.005306,0.523691
3,AB1_OTHER_VIRAL,Other viral diseases,LV964,0.006106,0.548143
4,AB1_OTHER_VIRAL,Other viral diseases,LV92,0.006565,0.560048


In [19]:
pd.testing.assert_frame_equal(
    data,
    data_again,
    check_dtype=False,
)

## Text format

In [20]:
# tsv format
output_file = remove_all_file_extensions(input_filepath).with_suffix(".tsv.gz")
output_file = OUTPUT_DIR / output_file.name
display(output_file)

PosixPath('/opt/code/data/gls/gls-summary-phenomexcan.tsv.gz')

In [21]:
data.to_csv(output_file, sep="\t", index=False, float_format="%.5e")

In [22]:
# testing
data2 = data.copy()
data2.index = list(range(0, data2.shape[0]))

data_again = pd.read_csv(output_file, sep="\t")
data_again.index = list(data_again.index)

In [23]:
data_again.shape

(4037817, 5)

In [24]:
data_again.head()

,phenotype,phenotype_desc,lv,pvalue,fdr
0,AB1_OTHER_VIRAL,Other viral diseases,LV736,0.004725,0.504339
1,AB1_OTHER_VIRAL,Other viral diseases,LV320,0.004848,0.508291
2,AB1_OTHER_VIRAL,Other viral diseases,LV366,0.005306,0.523691
3,AB1_OTHER_VIRAL,Other viral diseases,LV964,0.006106,0.548143
4,AB1_OTHER_VIRAL,Other viral diseases,LV92,0.006565,0.560048


In [25]:
pd.testing.assert_frame_equal(
    data2,
    data_again,
    check_categorical=False,
    check_dtype=False,
)